<a href="https://colab.research.google.com/github/Alan-Turing18this-Year/Automated-EDA-Narrator-Data-Quality-Scoring-Tool/blob/main/Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!rm -rf Automated-EDA-Narrator-Data-Quality-Scoring-Tool


/content


In [2]:
!git clone https://github.com/Alan-Turing18this-Year/Automated-EDA-Narrator-Data-Quality-Scoring-Tool.git
%cd Automated-EDA-Narrator-Data-Quality-Scoring-Tool
!ls


Cloning into 'Automated-EDA-Narrator-Data-Quality-Scoring-Tool'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 199 (delta 16), reused 0 (delta 0), pack-reused 170 (from 1)
Receiving objects: 100% (199/199), 70.03 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/Automated-EDA-Narrator-Data-Quality-Scoring-Tool
data		 dataset_uml.puml  presentation  requirements.txt  tests
dataset_uml.png  notebooks	   README.md	 src


In [3]:
!pip install -r requirements.txt


In [4]:
import sys
sys.path.append('src')


In [5]:
# Create sample CSV
import pandas as pd

data = {
    "id": [1,2,3,4,5,5,6],
    "name": ["Alice","Bob","Charlie","Denise","Ed","Ed","Frank"],
    "age": [34,29,45,None,23,23,200],
    "gender": ["Female","Male","Male","Female","Male","Male","Male"],
    "salary": [54000,48000,120000,60000,40000,40000,999999],
    "signup_date": ["2020-01-15","2019-11-02","2018-06-21","2021-03-11",
                    "2019-12-31","2019-12-31","2020-02-02"]
}
df = pd.DataFrame(data)
df.to_csv("data/sample.csv", index=False)


In [6]:
%%writefile src/orchestrator.py
from loader import DataLoader
from preprocessor import Preprocessor
from eda_analyzer import EDAAnalyzer
from quality_scorer import QualityScorer
from narrator import Narrator
from report_builder import ReportBuilder

class DatasetPipeline:
    def __init__(self, path):
        self.path = path        # CSV file path
        self.df = None          # DataFrame
        self.pre = None
        self.eda = None
        self.scores = None
        self.narrative = None
        self.report = None

    def run(self):
        # Load data
        self.df = DataLoader(self.path).load()

        # Preprocessing
        self.pre = Preprocessor(self.df).trim_strings(self.df.select_dtypes(include=['object']).columns.tolist())
        clean_df = self.pre.get()

        # EDA
        analyzer = EDAAnalyzer(clean_df)
        eda_results = analyzer.run_all()

        # Scoring
        scorer = QualityScorer(eda_results, df_len=len(clean_df))
        scorer.overall_score()
        scores = scorer.scores

        # Narration
        narrator = Narrator(eda_results, scores)
        narrative = narrator.generate()

        # Report
        builder = ReportBuilder(narrative, eda_results, scores)
        md = builder.to_markdown()

        # Save internal state
        self.eda = eda_results
        self.scores = scores
        self.narrative = narrative
        self.report = md

        return md


Overwriting src/orchestrator.py


In [7]:
import sys
sys.path.append('src')


In [8]:
from orchestrator import DatasetPipeline


In [9]:
pipeline = DatasetPipeline("data/sample.csv")
md = pipeline.run()
print(md)


# Automated EDA Report


## Narrative Insights


- Column 'id' has mean 3.71 and std 1.80.

- Column 'age' has mean 59.00 and std 69.56.

- Column 'salary' has mean 194571.29 and std 356233.09.

- Column 'age' has 1 missing values (14.29%).

- Column 'age' has 1 detected outliers.

- Column 'salary' has 1 detected outliers.

- Overall data quality: 81.67/100 — Good.


## Quality Scores


| Metric     |   Score |
|------------|---------|
| missing    |   97.62 |
| duplicates |   71.43 |
| outliers   |   57.14 |
| balance    |   90    |
| overall    |   81.67 |
